In [ ]:
from dataclasses import dataclass
import itertools
from typing import Any, Dict, Sequence, Union


NAME_DELIMITER = '-'
DISMISS_FORM = object()


def name(*parts: str) -> str:
    return NAME_DELIMITER.join(parts)


def dename(name: str) -> Sequence[str]:
    return name.split(NAME_DELIMITER)


class Form:
    NORMAL = '__normal__'
    # normal is implicit as there currently is no non-normal female form
    # with significant differences
    FEMALE = 'female'
    
    MEGA = 'mega'
    MEGA_X = name('mega', 'x')
    MEGA_Y = name('mega', 'y')
    GIGANTAMAX = 'gigantamax'
    ALOLA = 'alola'
    GALAR = 'galar'
    HISUI = 'hisui'


@dataclass
class PokemonForm:
    """Specifies an appearance of a pokemon."""
    
    ndex: int
    name: str
    color_only: bool = False

    @property
    def complete_name(self):
        if self.name == Form.NORMAL:
            return f'{self.ndex}'
        else:
            return name(str(self.ndex), self.name)


class NoNormal(tuple):
    ...


def to_kwargs(form_descriptor: Union[str, Dict[str, Any], ]):
    """Converts a form descriptor to kwargs suitable for a PokemonForm instance."""
    if isinstance(form_descriptor, str):
        kwargs = dict(name=form_descriptor)
    elif isinstance(form_descriptor, dict):
        assert all(isinstance(key, str) for key in form_descriptor.keys()), (
            'dict with non-string keys cannot be used as kwargs'
        )
        kwargs = form_descriptor
    return kwargs


def get_instances(ndex, form_descriptors: Union[str, tuple]):
    if not isinstance(form_descriptors, tuple):
        form_descriptors = (form_descriptors,)
    
    if not isinstance(form_descriptors, NoNormal):
        form_descriptors = (Form.NORMAL, *form_descriptors)

    return tuple(
        PokemonForm(ndex=ndex, **to_kwargs(form)) 
        for form in form_descriptors
    )
            


def get_forms(forms_by_ndex):
    default_forms = {
        ndex: (PokemonForm(ndex=ndex, name=Form.NORMAL), )
        for ndex in range(1, max(forms_by_ndex.keys()) + 1)
    }
    return {
        **default_forms,
        **{
            ndex: get_instances(ndex, forms)
            for ndex, forms in forms_by_ndex.items()
        },
    }


GET_FORM_SENTINEL = object()


def get_form(ndex: int, form_name: str, default = GET_FORM_SENTINEL) -> PokemonForm:
    forms = POKEMON_FORMS[ndex]
    matches = [f for f in forms if f.name == form_name]
    if len(matches) == 0 and default is not GET_FORM_SENTINEL:
        return default
    else:
        assert len(matches) == 1, f'got {len(matches)} matching forms instead 1 for #{name(str(ndex), form_name)}'
        return matches[0]


def no_normal(*forms):
    return NoNormal(forms)


def color_only(*forms):
    return tuple(
        {**to_kwargs(form), 'color_only': True}
        for form in forms
    )


# https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_with_form_differences

# NOTE: According to 
#  https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_with_gender_differences 
#  significant differences between male and female pokemon 
#  start happening from generation 5:
#  => 521, 592, 593, 668, 678, 876

POKEMON_FORMS = get_forms({
    3: (Form.MEGA, Form.GIGANTAMAX),  # Venusaur
    6: (Form.MEGA_X, Form.MEGA_Y, Form.GIGANTAMAX),  # Charizard
    9: (Form.MEGA, Form.GIGANTAMAX),  # Blastoise
    12: Form.GIGANTAMAX,  # Butterfree
    15: Form.MEGA,  # Beedrill
    18: Form.MEGA,  # Pidgeot
    19: Form.ALOLA,  # Rattata
    20: Form.ALOLA,  # Raticate
    25: (
        Form.GIGANTAMAX,
        'cosplay', 'cosplay-rock-star', 'cosplay-belle', 
        'cosplay-pop-star', 'cosplay-phd', 'cosplay-libre',
        'cap-original', 'cap-hoenn', 'cap-sinnoh', 'cap-unova', 
        'cap-kalos', 'cap-alola', 'cap-partner', 'cap-world',
    ),  # Pikachu
    26: Form.ALOLA,  # Raichu
    27: Form.ALOLA,  # Sandshrew
    28: Form.ALOLA,  # Sandslash
    37: Form.ALOLA,  # Vulpix
    38: Form.ALOLA,  # Ninetales
    50: Form.ALOLA,  # Diglett
    51: Form.ALOLA,  # Dugtrio
    52: (Form.ALOLA, Form.GALAR, Form.GIGANTAMAX),  # Meowth
    53: Form.ALOLA,  # Persian
    58: Form.HISUI,  # Growlithe
    65: Form.MEGA,  # Alakazam
    68: Form.GIGANTAMAX,  # Machamp
    74: Form.ALOLA,  # Geodude
    75: Form.ALOLA,  # Graveler
    76: Form.ALOLA,  # Golem
    77: Form.GALAR,  # Ponyta
    78: Form.GALAR,  # Rapidash
    79: Form.GALAR,  # Slowpoke
    80: (Form.MEGA, Form.GALAR),  # Slowbro
    83: Form.GALAR,  # Farfetch'd
    88: Form.ALOLA,  # Grimer
    89: Form.ALOLA,  # Muk
    94: (Form.MEGA, Form.GIGANTAMAX),  # Gengar
    99: Form.GIGANTAMAX,  # Kingler
    103: Form.ALOLA,  # Exeggutor
    105: Form.ALOLA,  # Marowak
    110: Form.GALAR,  # Weezing
    115: Form.MEGA,  # Kangaskhan
    122: Form.GALAR,  # Mr. Mime
    127: Form.MEGA,  # Pinsir
    130: Form.MEGA,  # Gyarados
    131: Form.GIGANTAMAX,  # Lapras
    133: Form.GIGANTAMAX,  # Eevee
    142: Form.MEGA,  # Aerodactyl
    143: Form.GIGANTAMAX,  # Snorlax
    144: Form.GALAR,  # Articuno
    145: Form.GALAR,  # Zapdos
    146: Form.GALAR,  # Moltres
    150: (Form.MEGA_X, Form.MEGA_Y),  # Mewtwo
    172: 'spiky-eared',  # Pichu
    181: Form.MEGA,  # Ampharos
    199: Form.GALAR,  # Slowking
    201: no_normal(
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 
        'h', 'i', 'j', 'k', 'l', 'm', 'n', 
        'o', 'p', 'q', 'r', 's', 't', 'u', 
        'v', 'w', 'x', 'y', 'z',
        'exclamation', 'question',
    ),  # Unown
    208: Form.MEGA,  # Steelix
    212: Form.MEGA,  # Scizor
    214: Form.MEGA,  # Heracross
    222: Form.GALAR,  # Corsola
    229: Form.MEGA,  # Houndoom
    248: Form.MEGA,  # Tyranitar
    254: Form.MEGA,  # Sceptile
    257: Form.MEGA,  # Blaziken
    260: Form.MEGA,  # Swampert
    263: Form.GALAR,  # Zigzagoon
    264: Form.GALAR,  # Linoone
    282: Form.MEGA,  # Gardevoir
    302: Form.MEGA,  # Sableye
    303: Form.MEGA,  # Mawile
    306: Form.MEGA,  # Aggron
    308: Form.MEGA,  # Medicham
    310: Form.MEGA,  # Manectric
    319: Form.MEGA,  # Sharpedo
    323: Form.MEGA,  # Camerupt
    334: Form.MEGA,  # Altaria
    351: ('sunny', 'rainy', 'snowy'),  # Castform
    354: Form.MEGA,  # Banette
    359: Form.MEGA,  # Absol
    362: Form.MEGA,  # Glalie
    373: Form.MEGA,  # Salamence
    376: Form.MEGA,  # Metagross
    380: Form.MEGA,  # Latias
    381: Form.MEGA,  # Latios
    382: 'primal',  # Kyogre
    383: 'primal',  # Groudon
    384: Form.MEGA,  # Rayquaza
    386: ('attack', 'defense', 'speed'),  # Deoxys
    412: no_normal('plant', 'sandy', 'trash'),  # Burmy
    413: no_normal('plant', 'sandy', 'trash'),  # Wormadam
    421: no_normal('overcast', 'sunshine'),  # Cherrim
    422: no_normal('east', 'west'),  # Shellos
    423: no_normal('east', 'west'),  # Gastrodon
    428: Form.MEGA,  # Lopunny
    445: Form.MEGA,  # Garchomp
    448: Form.MEGA,  # Lucario
    460: Form.MEGA,  # Abomasnow
    475: Form.MEGA,  # Gallade
    479: ('fan', 'frost', 'heat', 'mow', 'wash'),  # Rotom
    487: no_normal('altered', 'origin'),  # Giratina
    492: no_normal('land', 'sky'),  # Shaymin
    # TODO: COLOR ONLY
    493: (
        'bug', 'dark', 'dragon', 'electric', 'fighting', 'fire', 
        'flying', 'ghost', 'grass', 'ground', 'ice', 'fairy', 
        'poison', 'psychic', 'rock', 'steel', 'water',
    ),  # Arceus
    521: Form.FEMALE,  # Unfezant
    531: Form.MEGA,  # Audino
    550: no_normal('blue-striped', 'red-striped'),  # Basculin
    554: Form.GALAR,  # Darumaka
    555: ('zen', Form.GALAR, f'{Form.GALAR}-zen'),  # Darmanitan
    569: Form.GIGANTAMAX,  # Garbodor
    # TODO: COLOR ONLY
    585: no_normal('spring', 'summer', 'autumn', 'winter'),  # Deerling
    586: no_normal('spring', 'summer', 'autumn', 'winter'),  # Sawsbuck
    592: Form.FEMALE,  # Frillish
    593: Form.FEMALE,  # Jellicent
    628: Form.HISUI,  # Braviary
    641: no_normal('incarnate', 'therian'),  # Tornadus
    642: no_normal('incarnate', 'therian'),  # Thundurus
    643: 'overdrive',  # Reshiram
    644: 'overdrive',  # Zekrom
    645: no_normal('incarnate', 'therian'),  # Landorus
    646: (
        'black', 'black-overdrive',
        'white', 'white-overdrive',
    ),  # Kyurem
    647: no_normal('ordinary', 'resolute'),  # Keldeo
    648: no_normal('aria', 'pirouette'),  # Meloetta
    # TODO: COLOR ONLY
    649: ('burn', 'chill', 'douse', 'shock'),  # Genesect
    658: 'ash',  # Greninja
    # TODO: COLOR ONLY
    666: (
        'archipelago', 'continental', 'elegant', 'fancy', 'garden', 'high-plains', 
        'icy-snow', 'jungle', 'marine', 'meadow', 'modern', 'monsoon', 
        'ocean', 'poke-ball', 'polar', 'river', 'sandstorm', 'savanna', 'sun', 'tundra',
    ),  # Vivillon
    668: Form.FEMALE,  # Pyroar
    # TODO: COLOR ONLY
    669: no_normal('blue', 'orange', 'red', 'white', 'yellow'),  # Flabébé
    # TODO: COLOR ONLY except 'az'
    670: ('blue', 'orange', 'red', 'white', 'yellow', 'az'),  # Floette
    # TODO: COLOR ONLY
    671: ('blue', 'orange', 'red', 'white', 'yellow'),  # Florges
    676: (
        'dandy', 'debutante', 'diamond', 'heart', 'kabuki', 
        'la-reine', 'matron', 'pharaoh', 'star',
    ),  # Furfrou
    678: Form.FEMALE,  # Meowstic
    681: no_normal('blade', 'shield'),  # Aegislash
    # 710, 711 => size differences don't matter for us
    716: no_normal('active', 'neutral'),  #  Xerneas
    718: no_normal('cell', 'core', '10-percent', '50-percent', 'complete'),  # Zygarde
    719: Form.MEGA,  # Diancie
    720: no_normal('confined', 'unbound'),  # Hoopa
    741: no_normal('baile', 'pau', 'pom-pom', 'sensu'),  # Oricorio
    745: no_normal('dusk', 'midday', 'midnight'),  # Lycanroc
    746: no_normal('solo', 'school'),  # Wishiwashi
    773: (
        'bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 
        'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 
        'poison', 'psychic', 'rock', 'steel', 'water',
    ),  # Silvally
    774: no_normal(
        'meteor',
        *color_only(
            'core-blue', 'core-green', 'core-indigo', 'core-orange', 
            'core-red', 'core-violet', 'core-yellow',
        ), 
    ),  # Minior
    778: no_normal('busted', 'disguised'),  # Mimikyu
    791: 'radiant-sun',  # Solgaleo
    792: 'full-moon',  # Lunala
    800: ('dusk-mane', 'dawn-wings', 'ultra'),  # Necrozma
    # TODO: COLOR ONLY
    801: 'original-color',  # Magearna
    802: 'zenith',  # Marshadow
    809: Form.GIGANTAMAX,  # Melmetal
    812: Form.GIGANTAMAX,  # Rillaboom
    815: Form.GIGANTAMAX,  # Cinderace
    818: Form.GIGANTAMAX,  # Inteleon
    823: Form.GIGANTAMAX,  # Corviknight
    826: Form.GIGANTAMAX,  # Orbeetle
    834: Form.GIGANTAMAX,  # Drednaw
    839: Form.GIGANTAMAX,  # Coalossal
    841: Form.GIGANTAMAX,  # Flapple
    842: Form.GIGANTAMAX,  # Appletun
    844: Form.GIGANTAMAX,  # Sandaconda
    845: ('gorging', 'gulping'),  # Cramorant
    849: no_normal(Form.GIGANTAMAX, 'amped', 'low-key'),  # Toxtricity
    851: Form.GIGANTAMAX,  # Centiskorch
    854: no_normal('antique', 'phony'),  # Sinistea
    855: no_normal('antique', 'phony'),  # Polteageist
    858: Form.GIGANTAMAX,  # Hatterene
    861: Form.GIGANTAMAX,  # Grimmsnarl
    869: no_normal(
        Form.GIGANTAMAX,
        # TODO: COLOR ONLY
        *[
            f'{cream}-{sweet}' for cream, sweet in itertools.product(
                [
                    'vanilla-cream', 'ruby-cream', 'matcha-cream', 'mint-cream', 'lemon-cream', 
                    'salted-cream', 'ruby-swirl', 'caramel-swirl', 'rainbow-swirl',
                ],
                [
                    'strawberry-sweet', 'love-sweet', 'berry-sweet', 'clover-sweet', 'flower-sweet', 
                    'star-sweet', 'ribbon-sweet',
                ],
            )
        ],
    ),  # Alcremie
    875: no_normal('ice-face', 'noice-face'),  # Eiscue
    877: no_normal('full-belly', 'hangry'),  # Morpeko
    879: Form.GIGANTAMAX,  # Copperajah
    884: Form.GIGANTAMAX,  # Duraludon
    888: no_normal('hero-of-many-battles', 'crowned-sword'),  # Zacian
    889: no_normal('hero-of-many-battles', 'crowned-shield'),  # Zamazenta
    890: 'eternamax',  # Eternatus
    892: no_normal(Form.GIGANTAMAX, 'single-strike', 'rapid-strike'),  # Urshufi
    893: 'dada',  # Zarude
    898: ('ice-rider', 'shadow-rider'),  # Calyrex
})
import pprint; pprint.pprint(POKEMON_FORMS)

In [132]:
from abc import ABC, abstractmethod
import hashlib
import os
from pathlib import Path
import shutil
from typing import Dict, Iterable, Optional

from cairosvg import svg2png
from py7zr import unpack_7zarchive
import requests
from wand.image import Image


TMP_DIR = Path('../tmp/')


def with_stem(path: Path, stem) -> Path:
    """Polyfill function
    https://github.com/python/cpython/blob/56c1f6d7edad454f382d3ecb8cdcff24ac898a50/Lib/pathlib.py#L764-L766
    """
    return path.with_name(stem + path.suffix)

def readlink(path: Path) -> Path:
    """Polyfill function"""
#     return Path(os.readlink(str(path.resolve())))
    return Path(os.readlink(path))


def create_gif_from_frames(
    frames,
    filename: Path,
    delay: int = int(100/24),
) -> None:
    """https://stackoverflow.com/a/40088327/6928824"""

    with Image() as img:
        for frame in frames:
            img.sequence.append(frame)
        for frame in img.sequence:
            frame.delay = delay
#         # Create progressive delay for each frame
#         for cursor in range(len(frames)):
#             with img.sequence[cursor] as frame:
#                 frame.delay = delay * (cursor + 1)
        img.type = 'optimize'
        img.save(filename=filename.with_suffix('.gif'))


class PathDict(dict):
    
    @classmethod
    def with_prefix(cls, prefix: str, **kwargs):
        return cls({
            f'{prefix}{key}': value 
            for key, value in kwargs.items()
        })
    
    def __getitem__(self, path: Path):
        assert isinstance(path, Path), f'expected key to be a Path but got {type(path).__name__}'
        for key, val in self.items():
            if path.match(key) or path.with_suffix('').match(key):
                return val
        return None


class DataSource(ABC):
    checksum = None
    extra_ops = ()
    
    _renamed_files = set()
    
    def __init__(self, *, checksum=None, extra_ops=None):
        if checksum is not None:
            self.checksum = checksum
        if extra_ops is not None:
            self.extra_ops = extra_ops
    
    def run(self, force=False):
        self.tmp_dir.mkdir(parents=True, exist_ok=True)
        data_path = self.get(force)
        self.verify_checksum(data_path)
        self.process(data_path)
        self.arrange()
        self.do_extra_ops()
        self._renamed_files = self.rename_forms()
#         self.delete_unused_files()

    @abstractmethod
    def get(self, force):
        ...
    
    def verify_checksum(self, data_path):
        CHUNK_SIZE = 1024 * 64
        hash_sha256 = hashlib.sha256()
        with open(data_path, 'rb') as f:
            for chunk in iter(lambda: f.read(CHUNK_SIZE), b''):
                hash_sha256.update(chunk)

        checksum = hash_sha256.hexdigest()
        if self.checksum is not None:
            assert self.checksum == checksum, (
                f'invalid checksum. expected {self.checksum} but got {checksum}'
            )
        else:
            print(f'checksum for {data_path} is {checksum}')
    
    def process(self, archive):
        ...
    
    @property
    def tmp_dir(self):
        return TMP_DIR / self.__class__.__name__
    
    def arrange(self):
        ...
    
    def do_extra_ops(self):
        for src, dst in self.extra_ops:
            print('extra_op', src, dst)
            if dst is None:
                try:
                    if (TMP_DIR / src).is_dir():
                        shutil.rmtree(TMP_DIR / src)
                    else:
                        (TMP_DIR / src).unlink()
                except FileNotFoundError:
                    print('tried to delete', TMP_DIR / src)
            else:
                shutil.move(str(TMP_DIR / src), str(TMP_DIR / dst))
    
    def rename_forms(self) -> None:        
        files = set()

        assigned_forms = self.assign_forms()
        # NOTE: Sorting enhances value of printed operations 
        #  but is essential for having a deterministic order so that 
        #  the developer can solve issues with chained renames
        for filename in sorted(self.get_files()):
            if filename.is_symlink():
                print('dismissing detected symlink', filename, '=>', readlink(filename))
                filename.unlink()
            else:
                stem = filename.stem
                form = assigned_forms[filename]
                if form is None:
                    ndex = self.parse_ndex(stem)
                    forms = [
                        form 
                        for form in POKEMON_FORMS[ndex] 
                        if form.complete_name == stem
                    ]
                    assert len(forms) == 1, f'got {len(forms)} matching forms instead 1 for {filename}'
                    form = forms[0]

                if form is DISMISS_FORM:
                    print('dismissing', filename)
                    filename.unlink()
                else:

                    # Avoid unncessary renames
                    if form.complete_name != stem:
                        # print('rename:', filename, filename.parent / f'{form.complete_name}{filename.suffix}')
                        # filename.rename(filename.parent / f'{form.complete_name}{filename.suffix}')
                        rename_to = with_stem(filename, form.complete_name)
                        print('rename:', filename, rename_to)
                        filename.rename(rename_to)
                        files.add(rename_to)
                    else:
                        files.add(filename)
        return files

    def assign_forms(self) -> PathDict:
        return PathDict()
    
    @abstractmethod
    def get_files(self) -> Iterable[Path]:
        ...
    
    def parse_ndex(self, filename: str) -> int:
        ndex_str, *rest = dename(filename)
        return int(ndex_str)
    
    @property
    def renamed_filenames(self):
        return self._renamed_files.copy()
    
#     def delete_unused_files(self):
#         ...


class ArchiveDataSource(DataSource):
    
    def process(self, archive):
        shutil.unpack_archive(filename=archive, extract_dir=self.tmp_dir)

        
class RemoteDataSource(ArchiveDataSource):
    url = None
    
    def __init__(self, *, url=None, **kwargs):
        super().__init__(**kwargs)
        if url is not None:
            self.url = url
        assert self.url is not None, 'url is not defined'
    
    def get(self, force):
        download_dest = TMP_DIR / Path(self.url).name
        if not download_dest.exists() or force:
            print('fetching', self.url)
            response = requests.get(self.url)
            response.raise_for_status()
            with open(download_dest, 'wb') as f:
                f.write(response.content)
        return download_dest


class SpriteSetDataSource(RemoteDataSource):
    sprite_sets = {
        # 'pokemon/main-sprites/red-blue': {
        #     'dest': None,  # rename
        #     'glob': '*.png',
        # },
    }
    
    def arrange(self):
        """Moves sprite set folders into TMP_DIR."""
        
        for src, conf in self.sprite_sets.items():
            src = self.tmp_dir / src
            dest = TMP_DIR / conf.get('dest', src.name)
            if dest.exists():
                print('deleting existing', dest)
                shutil.rmtree(dest)
            shutil.copytree(src, dest)
        shutil.rmtree(self.tmp_dir)

    def get_files(self):
        for src, conf in self.sprite_sets.items():
            dest = TMP_DIR / conf.get('dest', Path(src).name)
            pattern = conf['glob']
            yield from dest.glob(pattern)


In [ ]:
class Gen1Veekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/generation-1.tar.gz'
    checksum = '2d0923f5abf1171b7e011b3ce9b879e8eee1fd56ec82dfbe597a2eafa63ca21c'
    sprite_sets = {
        'pokemon/main-sprites/red-blue': dict(glob='*.png'),
        'pokemon/main-sprites/red-green': dict(glob='*.png'),
        'pokemon/main-sprites/yellow': dict(glob='*.png'),
        'pokemon/main-sprites/yellow/gbc': dict(dest='yellow-gbc', glob='*.png'),
    }
    extra_ops = (
        ('red-blue/back', None),
        ('red-blue/gray', None),
        ('red-green/back', None),
        ('red-green/gray', None),
        ('yellow/back', None),
        ('yellow/gray', None),
        ('yellow/gbc', None),
    )


Gen1Veekun().run()

In [ ]:
class Gen2Veekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/generation-2.tar.gz'
    checksum = '1a01266008cf726df5d273da96ec3cbbbd3da0f17bfada4b0b153a4c92b4517a'
    sprite_sets = {
        'pokemon/main-sprites/gold': dict(glob='*.png'),
        'pokemon/main-sprites/silver': dict(glob='*.png'),
        'pokemon/main-sprites/crystal': dict(glob='*.png'),
        'pokemon/main-sprites/crystal/animated': dict(dest='crystal-animated', glob='*.gif'),
    }
    extra_ops = (
        ('gold/back', None),
        ('gold/shiny', None),
        ('silver/back', None),
        ('silver/shiny', None),
        ('crystal/animated', None),
        ('crystal/back', None),
        ('crystal/shiny', None),
        ('crystal-animated/shiny', None),
    )
    
    def assign_forms(self):
        return PathDict(**{
            'gold/201': DISMISS_FORM,
            'silver/201': DISMISS_FORM,
            'crystal/201': DISMISS_FORM,
            'crystal-animated/201': get_form(201, 'u'),
        })  


Gen2Veekun().run()

In [ ]:
class Gen3Veekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/generation-3.tar.gz'
    checksum = '15b733baf9ef91fbde3ae957edb4d2ba75615601a515b41590ab87043370319c'
    sprite_sets = {
        'pokemon/main-sprites/ruby-sapphire': dict(glob='*.png'),
        'pokemon/main-sprites/emerald': dict(glob='*.png'),
        'pokemon/main-sprites/emerald/animated': dict(dest='emerald-animated', glob='*.gif'),
        'pokemon/main-sprites/emerald/frame2': dict(dest='emerald-frame2', glob='*.png'),
        'pokemon/main-sprites/firered-leafgreen': dict(glob='*.png'),
    }
    extra_ops = (
        ('ruby-sapphire/back', None),
        ('ruby-sapphire/shiny', None),
        ('emerald/animated', None),
        ('emerald/frame2', None),
        ('emerald/shiny', None),
        ('emerald-animated/shiny', None),
        ('firered-leafgreen/back', None),
        ('firered-leafgreen/shiny', None),
    )
    
    def assign_forms(self):
        return PathDict(**{
            'ruby-sapphire/201': get_form(201, 'j'),
            'ruby-sapphire/386-normal': get_form(386, Form.NORMAL),
            'emerald/201': get_form(201, 'j'),
            'emerald/386-normal': get_form(386, Form.NORMAL),
            'emerald-animated/386-normal': get_form(386, Form.NORMAL),
            'emerald-frame2/201*': DISMISS_FORM,  # empty image
            'emerald-frame2/351*': DISMISS_FORM,  # empty image
            'emerald-frame2/386*': DISMISS_FORM,  # empty image
            'firered-leafgreen/386-normal': get_form(386, Form.NORMAL),
        })


Gen3Veekun().run()

In [ ]:
class Gen4Veekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/generation-4.tar.gz'
    checksum = 'b1b69463aac872b54adf56f1159e8e6d2dfcbbecb7d71c7ebf832fe44140da41'
    sprite_sets = {
        'pokemon/main-sprites/diamond-pearl': dict(glob='*.png'),
        'pokemon/main-sprites/diamond-pearl/frame2': dict(dest='diamond-pearl-frame2', glob='*.png'),
        'pokemon/main-sprites/platinum': dict(glob='*.png'),
        'pokemon/main-sprites/platinum/frame2': dict(dest='platinum-frame2', glob='*.png'),
        'pokemon/main-sprites/heartgold-soulsilver': dict(glob='*.png'),
        'pokemon/main-sprites/heartgold-soulsilver/frame2': dict(dest='heartgold-soulsilver-frame2', glob='*.png'),
    }
    extra_ops = (
        ('diamond-pearl/back', None),
        ('diamond-pearl/female', None),
        ('diamond-pearl/frame2', None),
        ('diamond-pearl/shiny', None),
        ('platinum/back', None),
        ('platinum/female', None),
        ('platinum/frame2', None),
        ('platinum/shiny', None),
        ('heartgold-soulsilver/back', None),
        ('heartgold-soulsilver/female', None),
        ('heartgold-soulsilver/frame2', None),
        ('heartgold-soulsilver/shiny', None),
    )
    
    def assign_forms(self):
        return PathDict(**{
            'diamond-pearl/201': get_form(201, 'a'),
            'diamond-pearl/386-normal': get_form(386, Form.NORMAL),
            'diamond-pearl/412': get_form(412, 'plant'),
            'diamond-pearl/413': get_form(413, 'plant'),
            'diamond-pearl/421': get_form(421, 'overcast'),
            'diamond-pearl/422': get_form(422, 'west'),
            'diamond-pearl/423': get_form(423, 'west'),
            'diamond-pearl/487': get_form(487, 'altered'),
            'diamond-pearl/492': get_form(492, 'land'),
            'diamond-pearl/493-normal': get_form(493, Form.NORMAL),
            'diamond-pearl/493-unknown': DISMISS_FORM,  # invalid form
            'diamond-pearl-frame2/201': DISMISS_FORM,  # empty image
            'diamond-pearl-frame2/386-normal': get_form(386, Form.NORMAL),
            'diamond-pearl-frame2/412': get_form(412, 'plant'),
            'diamond-pearl-frame2/413': get_form(413, 'plant'),
            'diamond-pearl-frame2/421': get_form(421, 'overcast'),
            'diamond-pearl-frame2/422': DISMISS_FORM,  # invalid form, seems mixed
            'diamond-pearl-frame2/423': DISMISS_FORM,  # invalid form, seems mixed
            'diamond-pearl-frame2/487': get_form(487, 'altered'),
            'diamond-pearl-frame2/492': get_form(492, 'land'),
            'diamond-pearl-frame2/493-normal': get_form(493, Form.NORMAL),
            'diamond-pearl-frame2/493-unknown': DISMISS_FORM,  # invalid form
            'platinum/201': get_form(201, 'a'),
            'platinum/386-normal': get_form(386, Form.NORMAL),
            'platinum/412': get_form(412, 'plant'),
            'platinum/413': get_form(413, 'plant'),
            'platinum/421': get_form(421, 'overcast'),
            'platinum/422': get_form(422, 'west'),
            'platinum/423': get_form(423, 'west'),
            'platinum/487': get_form(487, 'altered'),
            'platinum/492': get_form(492, 'land'),
            'platinum/493-normal': get_form(493, Form.NORMAL),
            'platinum/493-unknown': DISMISS_FORM,  # invalid form
            'platinum-frame2/201*': DISMISS_FORM,  # empty image
            'platinum-frame2/351*': DISMISS_FORM,  # empty image
            'platinum-frame2/386*': DISMISS_FORM,  # empty image
            'platinum-frame2/412': get_form(412, 'plant'),
            'platinum-frame2/413': get_form(413, 'plant'),
            'platinum-frame2/421': get_form(421, 'overcast'),
            'platinum-frame2/422': get_form(422, 'west'),
            'platinum-frame2/423': get_form(423, 'west'),
            'platinum-frame2/487': get_form(487, 'altered'),
            'platinum-frame2/492': get_form(492, 'land'),
            'platinum-frame2/493-normal': get_form(493, Form.NORMAL),
            'platinum-frame2/493-unknown': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver/172-beta': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver/201': get_form(201, 'a'),
            'heartgold-soulsilver/386-normal': get_form(386, Form.NORMAL),
            'heartgold-soulsilver/412': get_form(412, 'plant'),
            'heartgold-soulsilver/412-beta': DISMISS_FORM,
            'heartgold-soulsilver/413': get_form(413, 'plant'),
            'heartgold-soulsilver/421': get_form(421, 'overcast'),
            'heartgold-soulsilver/421-beta': DISMISS_FORM,
            'heartgold-soulsilver/422': get_form(422, 'west'),  # different animation frame than 422-west
            'heartgold-soulsilver/423': get_form(423, 'west'),  # different animation frame than 423-west
            'heartgold-soulsilver/487': get_form(487, 'altered'),
            'heartgold-soulsilver/492': get_form(492, 'land'),
            'heartgold-soulsilver/493-normal': get_form(493, Form.NORMAL),
            'heartgold-soulsilver/493-unknown': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver/egg': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver/egg-manaphy': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver/substitute': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver-frame2/201*': DISMISS_FORM,  # empty image
            'heartgold-soulsilver-frame2/386-normal': get_form(386, Form.NORMAL),
            'heartgold-soulsilver-frame2/412': get_form(412, 'plant'),
            'heartgold-soulsilver-frame2/413': get_form(413, 'plant'),
            'heartgold-soulsilver-frame2/421': get_form(421, 'overcast'),
            'heartgold-soulsilver-frame2/422': get_form(422, 'west'),
            'heartgold-soulsilver-frame2/423': get_form(423, 'west'),
            'heartgold-soulsilver-frame2/487': get_form(487, 'altered'),
            'heartgold-soulsilver-frame2/492': get_form(492, 'land'),
            'heartgold-soulsilver-frame2/493-normal': get_form(493, Form.NORMAL),
            'heartgold-soulsilver-frame2/493-unknown': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver-frame2/egg': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver-frame2/egg-manaphy': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver-frame2/substitute': DISMISS_FORM,  # no pokemon
        })


Gen4Veekun().run()

In [ ]:
class Gen5Veekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/generation-5.tar.gz'
    checksum = 'ee037a3319b2a6143c5c90f679be13a06126c2f5424e46023fe0f53d2631aa62'
    sprite_sets = {
        'pokemon/main-sprites/black-white': dict(glob='*.png'),
    }
    extra_ops = (
        ('black-white/back', None),
        ('black-white/female/521.png', 'black-white/521-female.png'),
        ('black-white/female/592.png', 'black-white/592-female.png'),
        ('black-white/female/593.png', 'black-white/593-female.png'),
        ('black-white/female', None),
        ('black-white/shiny', None),
    )
    
    def assign_forms(self):
        return PathDict(**{
            'black-white/0': DISMISS_FORM,  # no pokemon
            'black-white/201': get_form(201, 'a'),
            'black-white/386-normal': get_form(386, Form.NORMAL),
            'black-white/412': get_form(412, 'plant'),
            'black-white/413': get_form(413, 'plant'),
            'black-white/421': get_form(421, 'overcast'),
            'black-white/422': get_form(422, 'west'),
            'black-white/423': get_form(423, 'west'),
            'black-white/487': get_form(487, 'altered'),
            'black-white/492': get_form(492, 'land'),
            'black-white/493-normal': get_form(493, Form.NORMAL),
            'black-white/550': get_form(550, 'red-striped'),
            'black-white/555': get_form(555, Form.NORMAL),
            'black-white/555-standard': get_form(555, Form.NORMAL),
            'black-white/555-zen': get_form(555, 'zen'),
            'black-white/585': get_form(585, 'spring'),
            'black-white/586': get_form(586, 'spring'),
            'black-white/641': get_form(641, 'incarnate'),
            'black-white/642': get_form(642, 'incarnate'),
            'black-white/645': get_form(645, 'incarnate'),
            'black-white/647': get_form(647, 'ordinary'),
            'black-white/648': get_form(648, 'aria'),
            'black-white/female/521': get_form(521, Form.FEMALE),
            'black-white/female/592': get_form(592, Form.FEMALE),
            'black-white/female/593': get_form(593, Form.FEMALE),
            'black-white/egg': DISMISS_FORM,  # no pokemon
            'black-white/egg-manaphy': DISMISS_FORM,  # no pokemon
            'black-white/substitute': DISMISS_FORM,  # no pokemon
        })


gen5_veekun = Gen5Veekun()
gen5_veekun.run()
# import pprint; pprint.pprint(gen5_veekun.filenames)

In [ ]:
class IconsVeekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/pokemon-icons.tar.gz'
    checksum = 'f9850ce82d8e6e69c163112c47553458fd27805034217a5331a1ae12b2a1c8ac'
    sprite_sets = {
        'pokemon/icons': dict(glob='*.png'),
    }
    extra_ops = (
        ('icons/egg.png', None),
        ('icons/female/521.png', 'icons/521-female.png'),
        ('icons/female/592.png', 'icons/592-female.png'),
        ('icons/female/593.png', 'icons/593-female.png'),
        ('icons/female/668.png', 'icons/668-female.png'),
        ('icons/female/678.png', 'icons/678-female.png'),
        ('icons/female', None),
        ('icons/old', None),
        ('icons/right', None),
    )
    
    def assign_forms(self):
        return PathDict(**{
            'icons/25-cosplay': get_form(25, 'cosplay'),
            'icons/25-rock-star': get_form(25, 'cosplay-rock-star'),
            'icons/25-belle': get_form(25, 'cosplay-belle'),
            'icons/25-pop-star': get_form(25, 'cosplay-pop-star'),
            'icons/25-phd': get_form(25, 'cosplay-phd'),
            'icons/25-libre': get_form(25, 'cosplay-libre'),
            'icons/201': get_form(201, 'a'),
            'icons/386-normal': get_form(386, Form.NORMAL),
            'icons/412': get_form(412, 'plant'),
            'icons/413': get_form(413, 'plant'),
            'icons/421': get_form(421, 'overcast'),
            'icons/422': get_form(422, 'west'),
            'icons/423': get_form(423, 'west'),
            'icons/487': get_form(487, 'altered'),
            'icons/492': get_form(492, 'land'),
            'icons/493-*': DISMISS_FORM,  # invalid forms (equal normal)
            'icons/550': get_form(550, 'red-striped'),
            'icons/555': get_form(555, Form.NORMAL),
            'icons/555-standard': get_form(555, Form.NORMAL),
            'icons/555-zen': get_form(555, 'zen'),
            'icons/585': get_form(585, 'spring'),
            'icons/586': get_form(586, 'spring'),
            'icons/641': get_form(641, 'incarnate'),
            'icons/642': get_form(642, 'incarnate'),
            'icons/645': get_form(645, 'incarnate'),
            'icons/647': get_form(647, 'ordinary'),
            'icons/648': get_form(648, 'aria'),
            'icons/649-*': DISMISS_FORM,  # invalid forms (equal normal)
            'icons/666': get_form(666, 'meadow'),
            'icons/669': get_form(669, 'red'),
            'icons/670': get_form(670, 'red'),
            'icons/670-eternal': DISMISS_FORM,  # unknown form
            'icons/671': get_form(671, 'red'),
            'icons/676': get_form(676, Form.NORMAL),
            'icons/676-natural': get_form(676, Form.NORMAL),
            'icons/678-male': get_form(678, Form.NORMAL),
            'icons/678-female': get_form(678, Form.FEMALE),
            'icons/681': get_form(681, 'shield'),
            'icons/710-*': DISMISS_FORM,
            'icons/711-*': DISMISS_FORM,
            'icons/716': get_form(716, 'active'),
            'icons/718': get_form(718, '50-percent'),
            'icons/720': get_form(720, 'confined'),
            'icons/egg': DISMISS_FORM,
        })


icons_veekun = IconsVeekun()
icons_veekun.run()

In [ ]:
class SugimoriVeekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/pokemon-sugimori.tar.gz'
    checksum = '9dcb5ab803725db99ec235df72da9cc20e96ac843d88394cff95a6b0bb06da16'
    sprite_sets = {
        'pokemon/sugimori': dict(glob='*.png'),
    }
    extra_ops = (
        ('sugimori/female/521.png', 'sugimori/521-female.png'),
        ('sugimori/female/592.png', 'sugimori/592-female.png'),
        ('sugimori/female/593.png', 'sugimori/593-female.png'),
        ('sugimori/female/668.png', 'sugimori/668-female.png'),
        ('sugimori/female/', None),
    )

    def assign_forms(self):
        return PathDict(**{
            'sugimori/25-cosplay': get_form(25, 'cosplay'),
            'sugimori/25-rock-star': get_form(25, 'cosplay-rock-star'),
            'sugimori/25-belle': get_form(25, 'cosplay-belle'),
            'sugimori/25-pop-star': get_form(25, 'cosplay-pop-star'),
            'sugimori/25-phd': get_form(25, 'cosplay-phd'),
            'sugimori/25-libre': get_form(25, 'cosplay-libre'),
            'sugimori/201-f': get_form(201, 'f'),
            'sugimori/201': get_form(201, 'f'),
            'sugimori/386-normal': get_form(386, Form.NORMAL),
            'sugimori/412': get_form(412, 'plant'),
            'sugimori/413': get_form(413, 'plant'),
            'sugimori/421': get_form(421, 'sunshine'),
            'sugimori/422': get_form(422, 'east'),
            'sugimori/423': get_form(423, 'east'),
            'sugimori/487': get_form(487, 'altered'),
            'sugimori/492': get_form(492, 'land'),
            'sugimori/493-normal': get_form(493, Form.NORMAL),
            'sugimori/521-female': get_form(521, Form.FEMALE),
            'sugimori/550': get_form(550, 'red-striped'),
            'sugimori/555': get_form(555, Form.NORMAL),
            'sugimori/555-standard': get_form(555, Form.NORMAL),
            'sugimori/585': get_form(585, 'spring'),
            'sugimori/586': get_form(586, 'spring'),
            'sugimori/592-female': get_form(592, Form.FEMALE),
            'sugimori/593-female': get_form(593, Form.FEMALE),
            'sugimori/641': get_form(641, 'incarnate'),
            'sugimori/642': get_form(642, 'incarnate'),
            'sugimori/645': get_form(645, 'incarnate'),
            'sugimori/647': get_form(647, 'ordinary'),
            'sugimori/648': get_form(648, 'aria'),
            # TODO: 666 equals 666-meadow but is the actual sugimori image
            'sugimori/666': get_form(666, 'meadow'),
            'sugimori/668-female': get_form(668, Form.FEMALE),
            'sugimori/669': get_form(669, 'red'),
            'sugimori/670': get_form(670, 'red'),
            'sugimori/671': get_form(671, 'red'),
            'sugimori/676': get_form(676, Form.NORMAL),
            'sugimori/678': get_form(678, Form.NORMAL),
            'sugimori/678-female': get_form(678, Form.FEMALE),
            'sugimori/681': DISMISS_FORM,  # both forms in 1 image
            'sugimori/716': get_form(716, 'active'),
            'sugimori/718': get_form(718, '50-percent'),
            'sugimori/720': get_form(720, 'confined'),
        })


sugimori_veekun = SugimoriVeekun()
sugimori_veekun.run()

In [ ]:
class DreamWorldVeekun(SpriteSetDataSource):
    url = 'https://veekun.com/static/pokedex/downloads/pokemon-dream-world.tar.gz'
    checksum = 'eaaf06ea99e71e34d8710f5cfd4923b8cd4d62f44124930afd02bc17046b6057'
    sprite_sets = {
        'pokemon/dream-world': dict(glob='*.svg'),
    }
    extra_ops = (
        # 521-female not available
        ('dream-world/female/592.svg', 'dream-world/592-female.svg'),
        ('dream-world/female/593.svg', 'dream-world/593-female.svg'),
        ('dream-world/female/', None),
    )
    
    def run(self, force=False):
        super().run(force)
        self.svg2png()
    
    def assign_forms(self):
        return PathDict(**{
            'dream-world/201': get_form(201, 'a'),
            'dream-world/386-normal': get_form(386, Form.NORMAL),
            'dream-world/412': get_form(412, 'plant'),
            'dream-world/413': get_form(413, 'plant'),
            'dream-world/421': get_form(421, 'overcast'),
            'dream-world/422': get_form(422, 'west'),
            'dream-world/423': get_form(423, 'west'),
            'dream-world/487': get_form(487, 'altered'),
            'dream-world/492': get_form(492, 'land'),
            'dream-world/493-normal': get_form(493, Form.NORMAL),
            'dream-world/550': get_form(550, 'red-striped'),
            'dream-world/555': get_form(555, Form.NORMAL),
            'dream-world/555-standard': get_form(555, Form.NORMAL),
            'dream-world/585': get_form(585, 'spring'),
            'dream-world/586': get_form(586, 'spring'),
            'dream-world/592-female': get_form(592, Form.FEMALE),
            'dream-world/593-female': get_form(593, Form.FEMALE),
            'dream-world/641': get_form(641, 'incarnate'),
            'dream-world/642': get_form(642, 'incarnate'),
            'dream-world/645': get_form(645, 'incarnate'),
            'dream-world/647': get_form(647, 'ordinary'),
            'dream-world/648': get_form(648, 'aria'),
        })

    def svg2png(self):
        for filename in (TMP_DIR / 'dream-world').iterdir():
            with open(filename) as f:
                svg2png(
                    file_obj=f,
                    write_to=str(TMP_DIR / 'dream-world' / f'{filename.stem}.png'),
                )
            filename.unlink()


dream_world_veekun = DreamWorldVeekun()
dream_world_veekun.run()

In [ ]:
class BattlersDataSource(SpriteSetDataSource):
    url = 'https://www.mediafire.com/folder/mi31mvoxx98ij/3D_Battlers'
    checksum = 'a282265f827aaf309f08c1be7ea98726de14bca942823ea85e6d7c77338d1205'
    sprite_sets = {
        'Front': dict(dest='3d-battlers-animated', glob='*.png'),
    }
    extra_ops = (
        ('3d-battlers-animated/53_1.png', '3d-battlers-animated/053_1.png'),  # only image without leading zeros
        ('3d-battlers-animated/Female/521.png', '3d-battlers-animated/521-female.png'),
        ('3d-battlers-animated/Female/592.png', '3d-battlers-animated/592-female.png'),
        ('3d-battlers-animated/Female/593.png', '3d-battlers-animated/593-female.png'),
        ('3d-battlers-animated/Female/668.png', '3d-battlers-animated/668-female.png'),
        ('3d-battlers-animated/Female/678.png', '3d-battlers-animated/678-female.png'),
        ('3d-battlers-animated/Female/', None),
    )
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        try:
            shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
        except shutil.RegistryError as e:
            if '.7z is already registered for "7zip"' in str(e):
                pass
            else:
                raise
    
    def run(self, force=False):
        super().run(force)
        self.animations2frames()
    
    def get(self, force):
        # TODO: Implement automatic download
        if not (TMP_DIR / '3D Battlers [All].7z').exists():
            print('Please manually download the file from')
            print('  https://www.mediafire.com/folder/mi31mvoxx98ij/3D_Battlers')
            print('and save it as tmp/3D Battlers [All].7z')
        return TMP_DIR / '3D Battlers [All].7z'
    
    def parse_ndex(self, filename: str) -> int:
        return super().parse_ndex(filename.replace('_', '-'))

    def assign_forms(self):
        N = 807
        DIGITS = 3
        return PathDict.with_prefix(
            '3d-battlers-animated/', 
            **{
                **{
                    str(i).zfill(DIGITS): get_form(i, Form.NORMAL)
                    for i in range(1, N + 1)
                    if get_form(i, Form.NORMAL, default=None)
                },
                **{
                    '003_1': get_form(3, Form.MEGA),
                    '006_1': get_form(6, Form.MEGA_X),
                    '006_2': get_form(6, Form.MEGA_Y),
                    '009_1': get_form(9, Form.MEGA),
                    '015_1': get_form(15, Form.MEGA),
                    '018_1': get_form(18, Form.MEGA),
                    '019_1': get_form(19, Form.ALOLA),
                    '020_1': get_form(20, Form.ALOLA),
                    '026_1': get_form(26, Form.ALOLA),
                    '027_1': get_form(27, Form.ALOLA),
                    '028_1': get_form(28, Form.ALOLA),
                    '037_1': get_form(37, Form.ALOLA),
                    '038_1': get_form(38, Form.ALOLA),
                    '050_1': get_form(50, Form.ALOLA),
                    '051_1': get_form(51, Form.ALOLA),
                    '052_1': get_form(52, Form.ALOLA),
                    '053_1': get_form(53, Form.ALOLA),
                    '065_1': get_form(65, Form.MEGA),
                    '074_1': get_form(74, Form.ALOLA),
                    '075_1': get_form(75, Form.ALOLA),
                    '076_1': get_form(76, Form.ALOLA),
                    '080_1': get_form(80, Form.MEGA),
                    '088_1': get_form(88, Form.ALOLA),
                    '089_1': get_form(89, Form.ALOLA),
                    '094_1': get_form(94, Form.MEGA),
                    '103_1': get_form(103, Form.ALOLA),
                    '105_1': get_form(105, Form.ALOLA),
                    '115_1': get_form(115, Form.MEGA),
                    '127_1': get_form(127, Form.MEGA),
                    '130_1': get_form(130, Form.MEGA),
                    '142_1': get_form(142, Form.MEGA),
                    '150_1': get_form(150, Form.MEGA_X),
                    '150_2': get_form(150, Form.MEGA_Y),
                    '181_1': get_form(181, Form.MEGA),
                    '201': get_form(201, 'a'),
                    '201_1': get_form(201, 'b'),
                    '201_2': get_form(201, 'd'),
                    '201_3': get_form(201, 'c'),
                    '201_4': get_form(201, 'e'),
                    '201_5': get_form(201, 'f'),
                    '201_6': get_form(201, 'g'),
                    '201_7': get_form(201, 'h'),
                    '201_8': get_form(201, 'i'),
                    '201_9': get_form(201, 'j'),
                    '201_10': get_form(201, 'k'),
                    '201_11': get_form(201, 'l'),
                    '201_12': get_form(201, 'm'),
                    '201_13': get_form(201, 'n'),
                    '201_14': get_form(201, 'o'),
                    '201_15': get_form(201, 'p'),
                    '201_16': get_form(201, 'q'),
                    '201_17': get_form(201, 'r'),
                    '201_18': get_form(201, 's'),
                    '201_19': get_form(201, 't'),
                    '201_20': get_form(201, 'u'),
                    '201_21': get_form(201, 'v'),
                    '201_22': get_form(201, 'w'),
                    '201_23': get_form(201, 'x'),
                    '201_24': get_form(201, 'y'),
                    '201_25': get_form(201, 'z'),
                    '201_26': get_form(201, 'question'),
                    '201_27': get_form(201, 'exclamation'),
                    '208_1': get_form(208, Form.MEGA),
                    '212_1': get_form(212, Form.MEGA),
                    '214_1': get_form(214, Form.MEGA),
                    '229_1': get_form(229, Form.MEGA),
                    '248_1': get_form(248, Form.MEGA),
                    '254_1': get_form(254, Form.MEGA),
                    '257_1': get_form(257, Form.MEGA),
                    '260_1': get_form(260, Form.MEGA),
                    '282_1': get_form(282, Form.MEGA),
                    '302_1': get_form(302, Form.MEGA),
                    '303_1': get_form(303, Form.MEGA),
                    '306_1': get_form(306, Form.MEGA),
                    '308_1': get_form(308, Form.MEGA),
                    '310_1': get_form(310, Form.MEGA),
                    '319_1': get_form(319, Form.MEGA),
                    '323_1': get_form(323, Form.MEGA),
                    '334_1': get_form(334, Form.MEGA),
                    '351_1': get_form(351, 'sunny'),
                    '351_2': get_form(351, 'rainy'),
                    '351_3': get_form(351, 'snowy'),
                    '354_1': get_form(354, Form.MEGA),
                    '359_1': get_form(359, Form.MEGA),
                    '362_1': get_form(362, Form.MEGA),
                    '373_1': get_form(373, Form.MEGA),
                    '376_1': get_form(376, Form.MEGA),
                    '380_1': get_form(380, Form.MEGA),
                    '381_1': get_form(381, Form.MEGA),
                    '382_1': get_form(382, 'primal'),
                    '383_1': get_form(383, 'primal'),
                    '384_1': get_form(384, Form.MEGA),
                    '386': get_form(386, Form.NORMAL),
                    '386_1': get_form(386, 'attack'),
                    '386_2': get_form(386, 'defense'),
                    '386_3': get_form(386, 'speed'),
                    '412': get_form(412, 'plant'),
                    '412_1': get_form(412, 'sandy'),
                    '412_2': get_form(412, 'trash'),
                    '413': get_form(413, 'plant'),
                    '413_1': get_form(413, 'sandy'),
                    '413_2': get_form(413, 'trash'),
                    '421': get_form(421, 'overcast'),
                    '421_1': get_form(421, 'sunshine'),
                    '422': get_form(422, 'west'),
                    '422_1': get_form(422, 'east'),
                    '423': get_form(423, 'west'),
                    '423_1': get_form(423, 'east'),
                    '428_1': get_form(428, Form.MEGA),
                    '445_1': get_form(445, Form.MEGA),
                    '448_1': get_form(448, Form.MEGA),
                    '460_1': get_form(460, Form.MEGA),
                    '475_1': get_form(475, Form.MEGA),
                    '479': get_form(479, Form.NORMAL),
                    '479_1': get_form(479, 'heat'),
                    '479_2': get_form(479, 'wash'),
                    '479_3': get_form(479, 'frost'),
                    '479_4': get_form(479, 'fan'),
                    '479_5': get_form(479, 'mow'),
                    '487': get_form(487, 'altered'),
                    '487_1': get_form(487, 'origin'),
                    '492': get_form(492, 'land'),
                    '492_1': get_form(492, 'sky'),
                    '493': get_form(493, Form.NORMAL),
                    '493_1': get_form(493, 'fighting'),
                    '493_2': get_form(493, 'flying'),
                    '493_3': get_form(493, 'poison'),
                    '493_4': get_form(493, 'ground'),
                    '493_5': get_form(493, 'rock'),
                    '493_6': get_form(493, 'bug'),
                    '493_7': get_form(493, 'ghost'),
                    '493_8': get_form(493, 'steel'),
                    # does not exist
                    # '493_9': get_form(493, ''),
                    '493_10': get_form(493, 'fire'),
                    '493_11': get_form(493, 'water'),
                    '493_12': get_form(493, 'grass'),
                    '493_13': get_form(493, 'electric'),
                    '493_14': get_form(493, 'psychic'),
                    '493_15': get_form(493, 'ice'),
                    '493_16': get_form(493, 'dragon'),
                    '493_17': get_form(493, 'dark'),
                    '493_18': get_form(493, 'fairy'),
                    '521-female': get_form(521, Form.FEMALE),
                    '531_1': get_form(531, Form.MEGA),
                    '550': get_form(550, 'red-striped'),
                    '550_1': get_form(550, 'blue-striped'),
                    '555': get_form(555, Form.NORMAL),
                    '555_1': get_form(555, 'zen'),
                    '585': get_form(585, 'spring'),
                    '585_1': get_form(585, 'summer'),
                    '585_2': get_form(585, 'autumn'),
                    '585_3': get_form(585, 'winter'),
                    '586': get_form(586, 'spring'),
                    '586_1': get_form(586, 'summer'),
                    '586_2': get_form(586, 'autumn'),
                    '586_3': get_form(586, 'winter'),
                    '592-female': get_form(592, Form.FEMALE),
                    '593-female': get_form(593, Form.FEMALE),
                    '641': get_form(641, 'incarnate'),
                    '641_1': get_form(641, 'therian'),
                    '642': get_form(642, 'incarnate'),
                    '642_1': get_form(642, 'therian'),
                    '645': get_form(645, 'incarnate'),
                    '645_1': get_form(645, 'therian'),
                    '646': get_form(646, Form.NORMAL),
                    '646_1': get_form(646, 'white'),
                    '646_2': get_form(646, 'black'),
                    '647': get_form(647, 'ordinary'),
                    '647_1': get_form(647, 'resolute'),
                    '648': get_form(648, 'aria'),
                    '648_1': get_form(648, 'pirouette'),
                    '649': get_form(649, Form.NORMAL),
                    '649_1': get_form(649, 'shock'),
                    '649_2': get_form(649, 'burn'),
                    '649_3': get_form(649, 'shock'),
                    '649_4': get_form(649, 'douse'),
                    '666': get_form(666, 'meadow'),
                    '666_1': get_form(666, 'polar'),
                    '666_2': get_form(666, 'tundra'),
                    '666_3': get_form(666, 'continental'),
                    '666_4': get_form(666, 'garden'),
                    '666_5': get_form(666, 'elegant'),
                    '666_6': get_form(666, 'icy-snow'),
                    '666_7': get_form(666, 'modern'),
                    '666_8': get_form(666, 'marine'),
                    '666_9': get_form(666, 'archipelago'),
                    '666_10': get_form(666, 'high-plains'),
                    '666_11': get_form(666, 'sandstorm'),
                    '666_12': get_form(666, 'river'),
                    '666_13': get_form(666, 'monsoon'),
                    '666_14': get_form(666, 'savanna'),
                    '666_15': get_form(666, 'sun'),
                    '666_16': get_form(666, 'ocean'),
                    '666_17': get_form(666, 'jungle'),
                    '666_18': get_form(666, 'fancy'),
                    '666_19': get_form(666, 'poke-ball'),
                    '668-female': get_form(668, Form.FEMALE),
                    '669': get_form(669, 'red'),
                    '669_1': get_form(669, 'yellow'),
                    '669_2': get_form(669, 'orange'),
                    '669_3': get_form(669, 'blue'),
                    '669_4': get_form(669, 'white'),
                    '670': get_form(670, 'red'),
                    '670_1': get_form(670, 'yellow'),
                    '670_2': get_form(670, 'orange'),
                    '670_3': get_form(670, 'blue'),
                    '670_4': get_form(670, 'white'),
                    '671': get_form(671, 'red'),
                    '671_1': get_form(671, 'yellow'),
                    '671_2': get_form(671, 'orange'),
                    '671_3': get_form(671, 'blue'),
                    '671_4': get_form(671, 'white'),
                    '678-female': get_form(678, Form.FEMALE),
                    '681': get_form(681, 'shield'),
                    '681_1': get_form(681, 'blade'),
                    '710': DISMISS_FORM,
                    '710_1': DISMISS_FORM,
                    '710_2': DISMISS_FORM,
                    '710_3': get_form(710, Form.NORMAL),  # use biggest image only
                    '711': DISMISS_FORM,
                    '711_1': DISMISS_FORM,
                    '711_2': DISMISS_FORM,
                    '711_3': get_form(711, Form.NORMAL),  # use biggest image only
                    '716': get_form(716, 'active'),
                    '718': get_form(718, '50-percent'),
                    '718_1': get_form(718, '10-percent'),
                    '718_2': get_form(718, 'complete'),
                    '719_1': get_form(719, Form.MEGA),
                    '720': get_form(720, 'confined'),
                    '720_1': get_form(720, 'unbound'),
                    '741': get_form(741, 'baile'),
                    '741_1': get_form(741, 'pom-pom'),
                    '741_2': get_form(741, 'pau'),
                    '741_3': get_form(741, 'sensu'),
                    '745': get_form(745, 'midday'),
                    '745_1': get_form(745, 'midnight'),
                    '745_2': get_form(745, 'dusk'),
                    '746': get_form(746, 'solo'),
                    '746_1': get_form(746, 'school'),
                    '773': get_form(773, Form.NORMAL),
                    '773_1': get_form(773, 'fighting'),
                    '773_2': get_form(773, 'flying'),
                    '773_3': get_form(773, 'poison'),
                    '773_4': get_form(773, 'ground'),
                    '773_5': get_form(773, 'rock'),
                    '773_6': get_form(773, 'bug'),
                    '773_7': get_form(773, 'ghost'),
                    '773_8': get_form(773, 'steel'),
                    # does not exist
                    # '773_9': get_form(773, ''),
                    '773_10': get_form(773, 'fire'),
                    '773_11': get_form(773, 'water'),
                    '773_12': get_form(773, 'grass'),
                    '773_13': get_form(773, 'electric'),
                    '773_14': get_form(773, 'psychic'),
                    '773_15': get_form(773, 'ice'),
                    '773_16': get_form(773, 'dragon'),
                    '773_17': get_form(773, 'dark'),
                    '773_18': get_form(773, 'fairy'),
                    '774': get_form(774, 'meteor'),
                    '774_1': get_form(774, 'core-red'),
                    '774_2': get_form(774, 'core-orange'),
                    '774_3': get_form(774, 'core-yellow'),
                    '774_4': get_form(774, 'core-green'),
                    '774_5': get_form(774, 'core-blue'),
                    '774_6': get_form(774, 'core-indigo'),
                    '774_7': get_form(774, 'core-violet'),
                    '778': get_form(778, 'disguised'),
                    '778_1': get_form(778, 'busted'),
                    '800': get_form(800, Form.NORMAL),
                    '800_1': get_form(800, 'dusk-mane'),
                    '800_2': get_form(800, 'dawn-wings'),
                    '800_3': get_form(800, 'ultra'),
                    '801': get_form(801, Form.NORMAL),
                    '801_1': get_form(801, 'original-color'),
                },
            },
        )
    
    def animations2frames(self):
        for filename in self.renamed_filenames:
            print('animations2frames', filename)
            img = Image(filename=filename)
            assert (img.width / img.height).is_integer(), 'invalid/non-integer image ratio'
            frame_width, frame_height = img.height, img.height
            
            i = 0
            x = 0
            first_frame = None
            # frames = []
            
            for x in range(0, img.width, frame_width):
                i = x // frame_width
                frame = img[x:x+frame_width, 0:frame_height]
                if x == 0:
                    first_frame = frame
                else:
                    if frame == first_frame:
                        print('found cycle at frame =', i)
                        break
                # frames.append(frame)
                frame.format = 'png'
                frame.save(
                    filename=with_stem(filename, name(filename.stem, str(i))),
                )
            # create_gif_from_frames(frames, filename)
            filename.unlink()


battlers = BattlersDataSource()
battlers.run()